In [3]:
import pandas as pd
from pymongo import MongoClient

In [4]:
client = MongoClient('localhost', 27017)
db = client.TableView
collection = db["Disease2SNP"]

In [62]:
collection.count_documents({})

0

In [7]:
df = pd.read_csv("df_gwas_collection.csv")

df = df[df.columns[1:]]

In [8]:
df.head()

,RSID,trait,mapped_gene
0,rs141175086,Morning vs. evening chronotype,LINC01128
1,rs2272756,Skin reflectance (Melanin index),NR
2,rs115438739,California verbal learning test score,NOC2L
3,rs13303010,Pancreatic cancer,NOC2L
4,rs13303327,Heel bone mineral density,KLHL17


In [9]:
df = df.drop_duplicates()

In [10]:
import collections
disease2rsid = collections.defaultdict(list)
for i, r in df.iterrows():
    rsid = r.RSID
    trait = r.trait
    disease2rsid[trait] = list(set(disease2rsid[trait]) | set([rsid]))

In [63]:
def populate(disease2rsid):
    for d in disease2rsid:
        m = d
        trait = m.replace(".", "").replace(" ", "_")
        doc = {"trait": trait, "SNPs": disease2rsid[d]}
        collection.insert_one(doc)

In [64]:
populate(disease2rsid)

In [65]:
collection.count_documents({})

4251

In [24]:
[ x for x in disease2rsid.keys() if "cardiomyopathy" in x]

['Chagas cardiomyopathy in Tripanosoma cruzi seropositivity',
 'Idiopathic dilated cardiomyopathy']

In [75]:
collection.find_one({'trait': 'Chagas_cardiomyopathy_in_Tripanosoma_cruzi_seropositivity'})

{'_id': ObjectId('5ed708fa5bc8d0e49d17aa74'),
 'trait': 'Chagas_cardiomyopathy_in_Tripanosoma_cruzi_seropositivity',
 'SNPs': ['rs73199525',
  'rs35955841',
  'rs4493363',
  'rs6910233',
  'rs4149018',
  'rs2480054',
  'rs12319113',
  'rs78356356',
  'rs2038868',
  'rs7829987',
  'rs182503338',
  'rs10132760',
  'rs10468092',
  'rs10769783',
  'rs1356410',
  'rs2262909',
  'rs35131064',
  'rs12542743']}

In [60]:
collection.drop()

In [53]:
d = 'Chagas cyopathy in Tripanosoma cruzi seropositivity'
d.replace(".", " ").replace(" ", "_")
d

'Chagas cyopathy in Tripanosoma cruzi seropositivity'

In [5]:
call = db.command("dbstats")
database = call['db']
datasize = call['dataSize'] / 1024
objects = call['objects']
collections = call['collections']

print('\n')
print('Database:', str(database))
print('Objects:', str(objects))
print('Collections:', str(collections))
print('Size:', str(datasize) + 'Mb')
print('\n')



Database: TableView
Objects: 42496735
Collections: 3
Size: 19278359.534179688Mb




In [6]:
db.command("dbstats")

{'db': 'TableView',
 'collections': 3,
 'views': 0,
 'objects': 42496735,
 'avgObjSize': 464.5307495505243,
 'dataSize': 19741040163.0,
 'storageSize': 3678785536.0,
 'numExtents': 0,
 'indexes': 7,
 'indexSize': 5130600448.0,
 'scaleFactor': 1.0,
 'fsUsedSize': 2251274477568.0,
 'fsTotalSize': 2918592225280.0,
 'ok': 1.0}

In [13]:
19741040163/1024/1024/1024

18.385276350192726

In [10]:
dbSNP153_Annotation = db["dbSNP153_Annotation"]
db.command("collstats","dbSNP153_Annotation")

{'ns': 'TableView.dbSNP153_Annotation',
 'size': 19737771536.0,
 'count': 42492483,
 'avgObjSize': 464,
 'storageSize': 3676794880.0,
 'capped': False,
 'wiredTiger': {'metadata': {'formatVersion': 1},
  'creationString': 'access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=1),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none),block_allocation=best,block_compressor=snappy,cache_resident=false,checksum=on,colgroups=,collator=,columns=,dictionary=0,encryption=(keyid=,name=),exclusive=false,extractor=,format=btree,huffman_key=,huffman_value=,ignore_in_memory_cache_size=false,immutable=false,internal_item_max=0,internal_key_max=0,internal_key_truncate=true,internal_page_max=4KB,key_format=q,key_gap=10,leaf_item_max=0,leaf_key_max=0,leaf_page_max=32KB,leaf_value_max=64MB,log=(enabled=true),lsm=(auto_throttle=true,bloom=true,bloom_bit_count=16,bloom_config=,bloom_hash_count=8,bloom_oldest=false,chunk_count_limit=0,chunk_max=5GB,chunk_size=10MB,merg

In [12]:
19737771536.0 /1024/1024/1024

18.382232204079628